# Bluegrass Songbook Analytics Dashboard

Analytics for visitor engagement, user activity, and content curation.

In [ ]:
# Setup
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
from supabase import create_client, ClientOptions
import pandas as pd
import matplotlib.pyplot as plt

# Load credentials from .env
load_dotenv()

SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_SERVICE_KEY')

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError('Missing SUPABASE_URL or SUPABASE_SERVICE_KEY in .env')

# Create client with session management disabled (server-side pattern)
# This avoids session persistence and reduces credential leak risk
supabase = create_client(
    SUPABASE_URL,
    SUPABASE_KEY,
    options=ClientOptions(
        auto_refresh_token=False,
        persist_session=False,
    )
)
print('Connected to Supabase (service role, no session)')

## Visitor Statistics

In [ ]:
# Daily visitor stats
result = supabase.table('visitor_stats').select('*').order('date', desc=True).limit(30).execute()
df_visits = pd.DataFrame(result.data)

if not df_visits.empty:
    df_visits['date'] = pd.to_datetime(df_visits['date'])
    df_visits = df_visits.sort_values('date')
    
    print(f"Total unique visitors: {df_visits['unique_visitors'].sum():,}")
    print(f"Total page views: {df_visits['page_views'].sum():,}")
    print(f"\nLast 7 days:")
    display(df_visits.tail(7))
else:
    print('No visitor data yet')

In [ ]:
# Plot daily visitors
if not df_visits.empty:
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.bar(df_visits['date'], df_visits['page_views'], alpha=0.7, label='Page Views')
    ax.plot(df_visits['date'], df_visits['unique_visitors'], 'ro-', label='Unique Visitors')
    ax.set_xlabel('Date')
    ax.set_ylabel('Count')
    ax.set_title('Daily Traffic (Last 30 Days)')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## User Activity

In [ ]:
# Get all users with their activity counts
# Note: auth.users is not directly accessible, but we can count activity from our tables

# Song votes by user
votes = supabase.table('song_votes').select('user_id, song_id').execute()
df_votes = pd.DataFrame(votes.data)

# Genre suggestions by user
suggestions = supabase.table('genre_suggestions').select('user_id, song_id, raw_tag').execute()
df_suggestions = pd.DataFrame(suggestions.data)

# Tag votes by user
tag_votes = supabase.table('tag_votes').select('user_id, song_id, tag_name, vote_value').execute()
df_tag_votes = pd.DataFrame(tag_votes.data)

print(f"Total song votes: {len(df_votes)}")
print(f"Total genre suggestions: {len(df_suggestions)}")
print(f"Total tag votes: {len(df_tag_votes)}")

In [ ]:
# Top users by total activity
activity = {}

if not df_votes.empty:
    for user_id in df_votes['user_id']:
        activity[user_id] = activity.get(user_id, 0) + 1

if not df_suggestions.empty:
    for user_id in df_suggestions['user_id']:
        activity[user_id] = activity.get(user_id, 0) + 1

if not df_tag_votes.empty:
    for user_id in df_tag_votes['user_id']:
        activity[user_id] = activity.get(user_id, 0) + 1

if activity:
    df_activity = pd.DataFrame([
        {'user_id': uid, 'total_actions': count}
        for uid, count in activity.items()
    ]).sort_values('total_actions', ascending=False)
    
    print(f"Total active users: {len(df_activity)}")
    print(f"\nTop 10 most active users:")
    display(df_activity.head(10))
else:
    print('No user activity yet')

## Song Engagement

In [ ]:
# Most voted songs
if not df_votes.empty:
    top_voted = df_votes.groupby('song_id').size().sort_values(ascending=False).head(10)
    print("Top 10 most voted songs:")
    for song_id, count in top_voted.items():
        print(f"  {song_id}: {count} votes")
else:
    print('No song votes yet')

In [ ]:
# Most tagged songs (genre suggestions)
if not df_suggestions.empty:
    top_tagged = df_suggestions.groupby('song_id').size().sort_values(ascending=False).head(10)
    print("Top 10 most tagged songs:")
    for song_id, count in top_tagged.items():
        print(f"  {song_id}: {count} tag suggestions")
else:
    print('No genre suggestions yet')

## Tag Analytics

In [ ]:
# Most suggested tags
if not df_suggestions.empty:
    tag_counts = df_suggestions['raw_tag'].value_counts().head(20)
    print("Top 20 suggested tags:")
    for tag, count in tag_counts.items():
        print(f"  {tag}: {count}")
else:
    print('No tag suggestions yet')

In [ ]:
# Tag vote summary
if not df_tag_votes.empty:
    tag_vote_summary = df_tag_votes.groupby('tag_name').agg(
        upvotes=('vote_value', lambda x: (x == 1).sum()),
        downvotes=('vote_value', lambda x: (x == -1).sum()),
        net_score=('vote_value', 'sum')
    ).sort_values('net_score', ascending=False)
    
    print("Tag vote summary (top 15):")
    display(tag_vote_summary.head(15))
else:
    print('No tag votes yet')

## Visitor Retention

In [ ]:
# Visitor first/last seen analysis
visitors = supabase.table('visitors').select('visitor_id, first_seen, last_seen').execute()
df_visitors = pd.DataFrame(visitors.data)

if not df_visitors.empty:
    df_visitors['first_seen'] = pd.to_datetime(df_visitors['first_seen'])
    df_visitors['last_seen'] = pd.to_datetime(df_visitors['last_seen'])
    
    # Returning visitors (last_seen != first_seen)
    df_visitors['is_returning'] = df_visitors['last_seen'].dt.date != df_visitors['first_seen'].dt.date
    returning_count = df_visitors['is_returning'].sum()
    total_visitors = len(df_visitors)
    
    print(f"Total unique visitors: {total_visitors:,}")
    print(f"Returning visitors: {returning_count:,} ({100*returning_count/total_visitors:.1f}%)")
    print(f"One-time visitors: {total_visitors - returning_count:,}")
    
    # New visitors by day
    new_by_day = df_visitors.groupby(df_visitors['first_seen'].dt.date).size().tail(14)
    print(f"\nNew visitors per day (last 14 days):")
    for date, count in new_by_day.items():
        print(f"  {date}: {count}")
else:
    print('No visitor data yet')

## User Lists

In [ ]:
# User-created lists
try:
    lists = supabase.table('user_lists').select('id, name, user_id, created_at').execute()
    df_lists = pd.DataFrame(lists.data)
    
    if not df_lists.empty:
        print(f"Total user lists: {len(df_lists)}")
        print(f"Users with lists: {df_lists['user_id'].nunique()}")
        
        # Songs per list
        list_songs = supabase.table('list_songs').select('list_id, song_id').execute()
        df_list_songs = pd.DataFrame(list_songs.data)
        
        if not df_list_songs.empty:
            songs_per_list = df_list_songs.groupby('list_id').size()
            print(f"Total songs in lists: {len(df_list_songs)}")
            print(f"Average songs per list: {songs_per_list.mean():.1f}")
    else:
        print('No user lists yet')
except Exception as e:
    print(f'Could not fetch user lists: {e}')

## Behavioral Analytics

Event-based analytics tracking user interactions.

In [ ]:
# Check if analytics_events table exists and has data
try:
    events = supabase.table('analytics_events').select('*').limit(1).execute()
    print(f"Analytics events table accessible")
    
    # Count total events
    all_events = supabase.table('analytics_events').select('event_name', count='exact').execute()
    print(f"Total events tracked: {all_events.count:,}")
    
    # Event type breakdown
    events_sample = supabase.table('analytics_events').select('event_name, created_at').order('created_at', desc=True).limit(1000).execute()
    df_events = pd.DataFrame(events_sample.data)
    
    if not df_events.empty:
        event_counts = df_events['event_name'].value_counts()
        print(f"\nEvent breakdown (last 1000):")
        for event, count in event_counts.items():
            print(f"  {event}: {count}")
    else:
        print("\nNo events recorded yet")
except Exception as e:
    print(f"Analytics events table not yet available: {e}")
    print("Run the migration: supabase/migrations/20251231000000_analytics_events.sql")

In [ ]:
# Most viewed songs (from analytics)
try:
    views = supabase.table('analytics_events').select('properties').eq('event_name', 'song_view').limit(1000).execute()
    df_views = pd.DataFrame(views.data)
    
    if not df_views.empty:
        df_views['song_id'] = df_views['properties'].apply(lambda x: x.get('song_id', '') if x else '')
        df_views['source'] = df_views['properties'].apply(lambda x: x.get('source', 'unknown') if x else 'unknown')
        df_views = df_views[df_views['song_id'] != '']
        
        # Top viewed songs
        top_songs = df_views['song_id'].value_counts().head(20)
        print(f"Top 20 most viewed songs ({len(df_views)} total views):\n")
        for song_id, count in top_songs.items():
            print(f"  {song_id[:50]:50} {count:4}")
        
        # View sources
        source_counts = df_views['source'].value_counts()
        print(f"\nView sources:")
        for source, count in source_counts.items():
            pct = 100 * count / len(df_views)
            print(f"  {source}: {count} ({pct:.1f}%)")
    else:
        print("No song_view events yet")
except Exception as e:
    print(f"Could not analyze song views: {e}")

In [ ]:
# Export behavior - How do users use songs?
try:
    exports = supabase.table('analytics_events').select('properties').eq('event_name', 'export').limit(500).execute()
    df_exports = pd.DataFrame(exports.data)
    
    if not df_exports.empty:
        df_exports['type'] = df_exports['properties'].apply(lambda x: x.get('type', 'unknown') if x else 'unknown')
        
        export_counts = df_exports['type'].value_counts()
        total = len(df_exports)
        
        print(f"Export usage ({total} total):\n")
        for export_type, count in export_counts.items():
            pct = 100 * count / total
            bar = '█' * int(pct / 5)
            print(f"  {export_type:20} {count:4} ({pct:5.1f}%) {bar}")
    else:
        print("No export events yet")
        
    # Also check print events
    prints = supabase.table('analytics_events').select('properties', count='exact').eq('event_name', 'print').execute()
    if prints.count and prints.count > 0:
        print(f"\nPrint actions: {prints.count}")
except Exception as e:
    print(f"Could not analyze export behavior: {e}")

In [ ]:
# Song engagement - Time spent viewing songs
try:
    song_time = supabase.table('analytics_events').select('properties').eq('event_name', 'song_time').limit(500).execute()
    df_time = pd.DataFrame(song_time.data)
    
    if not df_time.empty:
        df_time['duration'] = df_time['properties'].apply(lambda x: x.get('duration_seconds', 0) if x else 0)
        df_time['song_id'] = df_time['properties'].apply(lambda x: x.get('song_id', '') if x else '')
        df_time = df_time[df_time['duration'] > 0]
        
        if not df_time.empty:
            print(f"Song viewing time analysis ({len(df_time)} sessions):\n")
            print(f"  Average time: {df_time['duration'].mean():.1f} seconds")
            print(f"  Median time: {df_time['duration'].median():.1f} seconds")
            print(f"  Max time: {df_time['duration'].max():.0f} seconds")
            
            # Time buckets
            bins = [0, 10, 30, 60, 120, 300, float('inf')]
            labels = ['<10s', '10-30s', '30-60s', '1-2min', '2-5min', '>5min']
            df_time['bucket'] = pd.cut(df_time['duration'], bins=bins, labels=labels)
            bucket_counts = df_time['bucket'].value_counts().sort_index()
            
            print(f"\nTime distribution:")
            for bucket, count in bucket_counts.items():
                pct = 100 * count / len(df_time)
                bar = '█' * int(pct / 5)
                print(f"  {bucket:8} {count:4} ({pct:5.1f}%) {bar}")
            
            # Most engaged songs
            song_avg_time = df_time.groupby('song_id')['duration'].agg(['mean', 'count'])
            song_avg_time = song_avg_time[song_avg_time['count'] >= 3]  # Min 3 views
            song_avg_time = song_avg_time.sort_values('mean', ascending=False).head(10)
            
            if not song_avg_time.empty:
                print(f"\nMost engaging songs (min 3 views):")
                for song_id, row in song_avg_time.iterrows():
                    print(f"  {song_id[:40]:40} {row['mean']:.0f}s avg ({row['count']:.0f} views)")
    else:
        print("No song_time events yet")
except Exception as e:
    print(f"Could not analyze song time: {e}")

In [ ]:
# Feature adoption - Which features are users using?
try:
    # Get all events from last 30 days
    cutoff = (datetime.now() - timedelta(days=30)).isoformat()
    events = supabase.table('analytics_events').select('event_name').gte('created_at', cutoff).execute()
    df_events = pd.DataFrame(events.data)
    
    if not df_events.empty:
        event_counts = df_events['event_name'].value_counts()
        
        # Group by feature category
        features = {
            'Search & Discovery': ['search', 'search_result_click', 'zero_results'],
            'Song Viewing': ['song_view', 'song_time'],
            'Display Options': ['transpose', 'display_mode', 'font_size', 'theme_toggle'],
            'Export': ['export', 'print'],
            'User Engagement': ['favorite_toggle', 'list_action', 'tag_vote', 'tag_suggest'],
            'Navigation': ['navigation', 'deep_link'],
            'Editor': ['editor_open', 'submission']
        }
        
        print("Feature adoption (last 30 days):\n")
        for category, event_names in features.items():
            cat_total = sum(event_counts.get(e, 0) for e in event_names)
            print(f"{category}: {cat_total:,} events")
            for event in event_names:
                count = event_counts.get(event, 0)
                if count > 0:
                    print(f"  - {event}: {count:,}")
            print()
    else:
        print("No events in last 30 days")
except Exception as e:
    print(f"Could not analyze feature adoption: {e}")

In [ ]:
# Filter usage - Which search filters are being used?
try:
    searches = supabase.table('analytics_events').select('properties').eq('event_name', 'search').limit(1000).execute()
    df_filters = pd.DataFrame(searches.data)
    
    if not df_filters.empty:
        filter_counts = {
            'artist:': 0,
            'tag:': 0,
            'chord:': 0,
            'prog:': 0,
            'key:': 0,
            'title:': 0,
            'lyrics:': 0,
            'composer:': 0,
        }
        
        for props in df_filters['properties']:
            if props:
                if props.get('has_artist'): filter_counts['artist:'] += 1
                if props.get('has_tag'): filter_counts['tag:'] += 1
                if props.get('has_chord'): filter_counts['chord:'] += 1
                if props.get('has_prog'): filter_counts['prog:'] += 1
                if props.get('has_key'): filter_counts['key:'] += 1
                if props.get('has_title'): filter_counts['title:'] += 1
                if props.get('has_lyrics'): filter_counts['lyrics:'] += 1
                if props.get('has_composer'): filter_counts['composer:'] += 1
        
        total_searches = len(df_filters)
        print(f"Filter usage across {total_searches} searches:\n")
        for filter_name, count in sorted(filter_counts.items(), key=lambda x: -x[1]):
            pct = 100 * count / total_searches if total_searches > 0 else 0
            bar = '█' * int(pct / 5)
            print(f"  {filter_name:12} {count:4} ({pct:5.1f}%) {bar}")
    else:
        print("No search events yet")
except Exception as e:
    print(f"Could not analyze filter usage: {e}")

In [ ]:
# Zero results - What can't users find?
try:
    zero_results = supabase.table('analytics_events').select('properties, created_at').eq('event_name', 'zero_results').order('created_at', desc=True).limit(100).execute()
    df_zero = pd.DataFrame(zero_results.data)
    
    if not df_zero.empty:
        df_zero['query'] = df_zero['properties'].apply(lambda x: x.get('query', '') if x else '')
        df_zero = df_zero[df_zero['query'] != '']
        
        if not df_zero.empty:
            zero_counts = df_zero['query'].value_counts().head(20)
            print(f"Top 20 queries with zero results ({len(df_zero)} total):")
            print("(These represent content gaps or search UX issues)\n")
            for query, count in zero_counts.items():
                print(f"  \"{query}\": {count}")
        else:
            print("No zero-result queries recorded")
    else:
        print("No zero_results events yet (good sign!)")
except Exception as e:
    print(f"Could not fetch zero_results events: {e}")

In [ ]:
# Top searches - What are users looking for?
try:
    searches = supabase.table('analytics_events').select('properties').eq('event_name', 'search').order('created_at', desc=True).limit(500).execute()
    df_searches = pd.DataFrame(searches.data)
    
    if not df_searches.empty:
        # Extract query from properties
        df_searches['query'] = df_searches['properties'].apply(lambda x: x.get('query', '').lower().strip() if x else '')
        df_searches = df_searches[df_searches['query'] != '']
        
        if not df_searches.empty:
            query_counts = df_searches['query'].value_counts().head(25)
            print("Top 25 search queries:")
            for query, count in query_counts.items():
                print(f"  \"{query}\": {count}")
        else:
            print("No search queries with text yet")
    else:
        print("No search events yet")
except Exception as e:
    print(f"Could not fetch search events: {e}")